Running boilerplate LDA models from gensim for topic modeling.
Inspired by: https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

In [1]:
import nltk
import spacy
from nltk.corpus import stopwords
import pandas as pd
import gensim
from gensim.utils import simple_preprocess

# COVID Hate Dataset

In [2]:
# load dataset and tokenize
covid_df = pd.read_csv('./../../data/original_datasets/covid_hate.csv')
covid_df['tokenized'] = covid_df['Text'].apply(lambda x: simple_preprocess(x))
covid_df.head()

,Tweet ID,Text,label,tokenized
0,1242553623260868608,Are we still allowed to quote ancient Chinese ...,0,"[are, we, still, allowed, to, quote, ancient, ..."
1,1246508137638580225,@mamacat2u @VBeltiz More power to you! This C...,0,"[mamacat, vbeltiz, more, power, to, you, this,..."
2,1233468243534372865,"CNBC: WHO, Tedros reiterated that the virus co...",0,"[cnbc, who, tedros, reiterated, that, the, vir..."
3,1243626072387747841,"""The heightened racism experienced by Asian co...",1,"[the, heightened, racism, experienced, by, asi..."
4,1225611530978217989,Coronavirus and Nepali in China: KP Oli has di...,0,"[coronavirus, and, nepali, in, china, kp, oli,..."


In [3]:
# combine all the tokenized texts into one list
all_tokenized_text = []
for i in range(len(covid_df)):
    all_tokenized_text.extend(covid_df['tokenized'][i])
print(len(all_tokenized_text))

59841


In [4]:
# build bigram and trigram models
bigram = gensim.models.Phrases(all_tokenized_text, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[all_tokenized_text], threshold=100)
bigram_maker = gensim.models.phrases.Phraser(bigram)
trigram_maker = gensim.models.phrases.Phraser(trigram)

In [5]:
# remove stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
all_tokenized_text = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in all_tokenized_text]

In [6]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# form bigrams
all_tokenized_text_bi = [bigram_maker[doc] for doc in all_tokenized_text]
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
all_tokenized_text_bi = lemmatization(all_tokenized_text_bi)
# drop all the empty lists
all_tokenized_text_bi = [doc for doc in all_tokenized_text_bi if len(doc) > 0]
print(all_tokenized_text_bi[:20])

[['still'], ['allow'], ['quote'], ['ancient'], ['chinese'], ['proverb'], ['racist'], ['racismisavirus'], ['mamacat'], ['power'], ['chinese'], ['virus'], ['thing'], ['really'], ['show'], ['crazy'], ['low'], ['iq'], ['people'], ['go']]


In [8]:
dictionary = gensim.corpora.Dictionary(all_tokenized_text_bi)
corpus = [dictionary.doc2bow(doc) for doc in all_tokenized_text_bi]

In [9]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, random_state=100, update_every=1, chunksize=100, passes=10, per_word_topics=True)

In [10]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda.print_topics(num_topics=10, num_words=10))

[(0,
  '0.192*"fuck" + 0.114*"racism" + 0.070*"many" + 0.055*"back" + '
  '0.023*"really" + 0.014*"right" + 0.000*"world" + 0.000*"say" + 0.000*"kill" '
  '+ 0.000*"ccp"'),
 (1,
  '0.458*"coronavirus" + 0.086*"go" + 0.045*"iamnotavirus" + 0.023*"keep" + '
  '0.015*"asianamerican" + 0.004*"bring" + 0.000*"world" + 0.000*"say" + '
  '0.000*"kill" + 0.000*"ccp"'),
 (2,
  '0.273*"call" + 0.033*"community" + 0.030*"patient" + 0.025*"line" + '
  '0.000*"world" + 0.000*"say" + 0.000*"kill" + 0.000*"ccp" + 0.000*"fucking" '
  '+ 0.000*"trump"'),
 (3,
  '0.508*"chinese" + 0.041*"show" + 0.036*"year" + 0.000*"world" + 0.000*"say" '
  '+ 0.000*"kill" + 0.000*"ccp" + 0.000*"fucking" + 0.000*"trump" + '
  '0.000*"come"'),
 (4,
  '0.340*"covid" + 0.184*"get" + 0.004*"turn" + 0.000*"world" + 0.000*"say" + '
  '0.000*"kill" + 0.000*"ccp" + 0.000*"fucking" + 0.000*"come" + '
  '0.000*"trump"'),
 (5,
  '0.351*"virus" + 0.204*"asian" + 0.146*"country" + 0.000*"world" + '
  '0.000*"say" + 0.000*"kill" + 0

In [11]:
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda, texts=all_tokenized_text_bi, dictionary=dictionary, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score:', coherence_lda)

Coherence Score: -22.105412822786864


UMass Measure: https://mimno.infosci.cornell.edu/papers/mimno-semantic-emnlp.pdf

In [14]:
import pickle 
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(lda, corpus, dictionary)
LDAvis_prepared


C:\Users\Kai\miniconda3\envs\cs6471\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.323432  0.009078       1        1  10.994260
8      0.050999  0.278894       2        1  10.753516
1      0.045997 -0.128142       3        1  10.689990
3      0.038973 -0.038195       4        1  10.124780
7      0.036523 -0.029612       5        1   9.899043
4      0.034535 -0.024810       6        1   9.825907
0      0.031630 -0.019789       7        1   9.748677
9      0.028110 -0.015664       8        1   9.399333
2      0.028171 -0.015717       9        1   9.293967
6      0.028495 -0.016042      10        1   9.270527, topic_info=             Term         Freq        Total Category  logprob  loglift
43             co  1753.000000  1753.000000  Default  30.0000  30.0000
4         chinese  1539.000000  1539.000000  Default  29.0000  29.0000
62    coronavirus  1465.000000  1465.000000  Default  28.0000  28.0000
10          virus  1157.000000  1157.000000  Default  27.0000  27.0000
60          covid   999.000000   999.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
292        spread     0.287653     2.330619  Topic10  -9.1731   0.2862
532          even     0.287530     2.329636  Topic10  -9.1736   0.2862
184        infect     0.287779     2.331983  Topic10  -9.1727   0.2860
357          give     0.287612     2.330717  Topic10  -9.1733   0.2860
1357     thousand     0.287816     2.332380  Topic10  -9.1726   0.2860

[1124 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1        10  0.982970           allow
80       10  0.996356        american
47        1  0.997131           asian
61        3  0.961287   asianamerican
67        7  0.987382            back
66        3  0.810661           bring
198       9  0.996819            call
4         4  0.998600         chinese
43        2  0.998520              co
48        9  0.983661       community
62        3  0.998557     coronavirus
82        1  0.994867         country
60        6  0.998322           covid
85       10  0.966784            fake
35        2  0.983359            fear
27        7  0.996487            fuck
25        6  0.995629             get
18        3  0.992686              go
79        2  0.980580            hate
71       10  0.985786            hear
70        3  0.985447    iamnotavirus
68        3  0.976545            keep
28        5  0.672578           leave
22        9  0.967785            line
50        7  0.989514            many
33       10  0.992795        pandemic
75        9  0.973869         patient
17        5  0.996876          people
45        7  0.994046          racism
7         8  0.994487  racismisavirus
6         5  0.997647          racist
12        7  0.970972          really
40        7  0.954740           right
143       8  0.996834            shit
13        4  0.983342            show
11        2  0.983222           thing
32        6  0.882940            turn
10        1  0.998029           virus
78        4  0.978596            year, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 9, 2, 4, 8, 5, 1, 10, 3, 7])